In [1]:
# Libraries for loading in data and building features
import numpy as np
import pandas as pd
# from docx import Document
from modules import * #contains functions used in common with processing election and IRS data
import os

In [2]:
# Establish file paths for IRS data and the formatted vote totals for house races (See various State .py files)
irs_folder_path = '../data/irs_data/'
house_data_path = '../data/formatted_house_totals/'

# Using the OS library, create lists of all files
# There folders must be kept tidy, or incorrect data will be read
irs_files = [file for file in os.listdir(irs_folder_path) if os.path.isfile(os.path.join(irs_folder_path, file))]
PA_house_files = [file for file in os.listdir(house_data_path) if os.path.isfile(os.path.join(house_data_path, file)) and file.startswith('PA')]


In [3]:
PA_house = []

In [4]:
irs_data = []

In [5]:
PA_irs_list = []

In [6]:
for i in PA_house_files:
    state = i[0:2] #first two letters of the filename, corresponding to the state abbv.
    year = i[5:7] #two digit year 
    
    # Import house file
    house = pd.read_csv(f'{house_data_path}{i}')
    
    # Import IRS file corresponding to the year obtained from the filename of the house file
    # See 'IRS_Processing' .py file for how these are created
    irs = pd.read_csv(f'{irs_folder_path}irs_count_20{year}_f.csv')
    
    # Get IRS data only for the relevant state (this is done to ensure that states with identicle county names do not have their data mixed)
    irs_state = irs.loc[irs['STATE']==state]
    irs_county = irs_state['COUNTYNAME'].tolist()
    irs_county = [i for i in remove_keyword(irs_county,'county')] # format to match the House data
    
    #These two counties are misnamed in the IRS data
    irs_misnamed = {'de witt':'dewitt','jo daviess':'jodaviess'}
    # This '.get's (haha) the correct county names
    cor_counties = [irs_misnamed.get(item, item) for item in irs_county]
    
    irs_copy = irs_state.copy() #avoid setting a value on a copy of a slice
    irs_copy['COUNTYNAME'] = cor_counties

    irs_merge = house.merge(irs_copy, left_on='County', right_on='COUNTYNAME')
    
    name = f"{state}_{year}_irs"
    
    PA_house.append(house)
    
    irs_data.append(irs_state)
    
    PA_irs_list.append((name, irs_merge))

In [ ]:
PA_irs_list[1][1]

In [ ]:
PA_house[0]

In [ ]:
PA_counties_2012

In [41]:
PA_counties_2012 = PA_house[0]['County'].tolist()

In [42]:
misnamed_counties = []

In [43]:
misnamed_counties_irs = []

In [44]:
PA_counties_2012 = PA_house[0]['County'].tolist()

for i in PA_house:
    counties = i['County'].tolist()
    for c in counties:
        if c not in PA_counties_2012:
            misnamed_counties.append(c)
    if len (misnamed_counties) > 0:
        print(misnamed_counties)

In [45]:
for i in irs_data:
    counties = i['COUNTYNAME']
    
    irs_county = [i for i in remove_keyword(counties,'county')]
    irs_county = [i for i in remove_keyword(irs_county,'count')]
    irs_county = [i for i in remove_keyword(irs_county,'cou')]
    for county in irs_county:
        if county not in PA_counties_2012:
            misnamed_counties_irs.append(county)

In [ ]:
from functools import reduce

# Assuming PA_house is a list of dataframes
PA_counties_2012 = PA_house[0]['County'].tolist()

misnamed_counties = reduce(
    lambda acc, df: acc + list(filter(lambda c: c not in PA_counties_2012, df['County'].tolist())),
    PA_house,
    []
)

if len(misnamed_counties) > 0:
    print(misnamed_counties)


In [46]:
misnamed_counties

[]

In [47]:
misnamed_counties_irs

[]